In [15]:
from dotenv import load_dotenv
load_dotenv()
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI
import os
import json
import getenv

In [16]:
OPENROUTER_API_KEY="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"


In [17]:
# Initialize Pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# Create a Pinecone index
pc.create_index(
    name="rag",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'd25ca2bc082f72ae77540f638d7da1dc', 'Date': 'Sat, 24 Aug 2024 06:58:56 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [22]:
# Load the review data
data = json.load(open("reviews.json"))
data

{'reviews': [{'Professor_Name': 'Kevin Parent',
   'Department': 'Accounting department',
   'Quality': 1,
   'Difficulty': 4,
   'Class_Name': 'MGCR211',
   'Comment': 'for my beginning at McGill university in accounting, this teacher made it disaster. he is very difficult to understand, dont answer questions. dont take his class'},
  {'Professor_Name': 'Kevin Parent',
   'Department': 'Accounting department',
   'Quality': 1,
   'Difficulty': 1,
   'Class_Name': 'MGCR211',
   'Comment': 'I once decided to chart how often he used the phrases "these are things you need to be aware of" "was that clear" "is this ok" "obviously" which he uses to string sentences that make up the entire lecture, while reading slides for 3 hrs, a page later, gave up and stopped attending class and studied at home. he\'s THE worst. how is he a McGill proff?'},
  {'Professor_Name': 'Kevin Parent',
   'Department': 'Accounting department',
   'Quality': 2,
   'Difficulty': 3,
   'Class_Name': 'MGCR211',
   'Co

In [19]:
processed_data = []
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=OPENROUTER_API_KEY,
)

In [24]:
# Create embeddings for each review
for review in data["reviews"]:
    response = client.embeddings.create(
        input=review['Comment'], model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append(
        {
            "values": embedding,
            "id": review["Professor_Name"],
            "metadata":{
                "review": review["Comment"],
                "subject": review["Department"],
                "stars": review["Quality"],
            }
        }
    )


APIStatusError: Error code: 405

In [ ]:

# Insert the embeddings into the Pinecone index
index = pc.Index("rag")
upsert_response = index.upsert(
    vectors=processed_data,
    namespace="ns1",
)
print(f"Upserted count: {upsert_response['upserted_count']}")
